In [ ]:
#| default_exp common

# foundation / primatives

> App setup with header imports and default settings

In [ ]:
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from itertools import product
from enum import Enum, auto
from fastlucide import *

### Script setup

In [ ]:
# For some reason the text tailwind classes are not being properly generated
text_css = Style("""
.text-muted-foreground { color: var(--muted-foreground); }
.text-foreground { color: var(--foreground); }
.hover\\:text-foreground:hover { color: var(--foreground); }
""")

In [ ]:
theme_script = Script("""
(() => {
  const stored = localStorage.getItem('themeMode');
  const dark = stored ? stored === 'dark' : matchMedia('(prefers-color-scheme: dark)').matches;
  if (dark) document.documentElement.classList.add('dark');
  
  document.addEventListener('basecoat:theme', (e) => {
    const mode = e.detail?.mode || (document.documentElement.classList.contains('dark') ? 'light' : 'dark');
    const isDark = mode === 'dark';
    document.documentElement.classList.toggle('dark', isDark);
    localStorage.setItem('themeMode', isDark ? 'dark' : 'light');
  });
})();
""")

ss = SvgSprites('l-')

deps = {
    'scripts': [
        'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4',
        'https://cdn.jsdelivr.net/npm/lit@3/dist/index.js',
        'https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/js/core.iife.js',
        'https://cdn.jsdelivr.net/npm/franken-ui@2.0.0/dist/js/icon.iife.js',
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/all.min.js'
    ],
    'links': [
        'https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/basecoat.cdn.min.css'
    ]
}

def make_hdrs(deps):
    scripts = tuple(Script(src=url) for url in deps['scripts'])
    links = tuple(Link(rel='stylesheet', href=url) for url in deps['links'])
    return (theme_script, ss) + scripts + links + (text_css,)

hdrs = make_hdrs(deps)

Can I put the sprite sheet into headers? https://answerdotai.github.io/fastlucide/index.html.md

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->



<details class='tool-usage-details'>

```json
{
  "id": "toolu_012NymN9V7XxhuFwcAeXib8N",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://answerdotai.github.io/fastlucide/index.html.md"
    }
  },
  "result": "'# fastlucide\\n\\n\\n<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->\\n\\nA convenient FastH<TRUNCATED>"
}
```

</details>

Yes, you can! The `SvgSprites` object (your `ss`) needs to be rendered somewhere in your page's DOM. Putting it in the headers would work perfectly.

You could add it to your `hdrs` tuple like this:

```python
hdrs = make_hdrs(deps) + (ss,)
```

This way the sprite sheet will be available throughout your app for all the icons you reference. Does that answer your question, or would you like to explore other placement options?

### Fasthtml

In [ ]:
app = FastHTML(hdrs=hdrs, session_cookie="mysession")
rt = app.route

In [ ]:
srv = JupyUvi(app)
# srv.stop()

In [ ]:
def get_preview(app): return partial(HTMX, app=app, host=None, port=None)
p = get_preview(app)

In [ ]:
c = Div(
    fh.Button('Hey there', cls='btn-outline'),
)
p(c)

In [ ]:
basecoat_components = [
    'Accordion',
    'Alert',
    'Alert Dialog',
    'Avatar',
    'Badge',
    'Breadcrumb',
    'Button',
    'Button Group',
    'Card',
    'Checkbox',
    'Command',
    'Combobox',
    'Dialog',
    'Dropdown Menu',
    'Empty',
    'Field',
    'Form',
    'Input',
    'Input Group',
    'Item',
    'Kbd',
    'Label',
    'Pagination',
    'Popover',
    'Progress',
    'Radio Group',
    'Select',
    'Sidebar',
    'Skeleton',
    'Slider',
    'Spinner',
    'Switch',
    'Table',
    'Tabs',
    'Textarea',
    'Theme Switcher',
    'Toast',
    'Tooltip'
]

In [ ]:
def slugify(s):
    return s.lower().replace(" ", "-")

In [ ]:
slugify("This content")

'this-content'

In [ ]:
# To easily preview items in a larger container
def Window(*args):
    return Div(*args, cls="w-full h-96 flex items-center justify-center")

### Accordion

Please create a fashtml version of Accordion based on the example provided in the docs.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a complete FastHTML implementation of the Accordion:

```python
def AccordionItem(title, content, **kwargs):
    return Details(
        Summary(
            H2(
                title,
                UkIcon("chevron-down", height=24, width=24, 
                       cls="text-muted-foreground pointer-events-none size-4 shrink-0 translate-y-0.5 transition-transform duration-200 group-open:rotate-180"),
                cls="flex flex-1 items-start justify-between gap-4 py-4 text-left text-sm font-medium hover:underline"
            ),
            cls="w-full focus-visible:border-ring focus-visible:ring-ring/50 focus-visible:ring-[3px] transition-all outline-none rounded-md"
        ),
        Section(
            P(content, cls="text-sm"),
            cls="pb-4"
        ),
        cls="group border-b last:border-b-0",
        **kwargs
    )

def Accordion(*items, **kwargs):
    return Div(
        Section(*items, cls="accordion"),
        **kwargs
    )
```

You can test it like this:

```python
p(
    Window(
      Accordion(
          AccordionItem("Is it accessible?", "Yes. It adheres to the WAI-ARIA design pattern."),
          AccordionItem("Is it styled?", "Yes. It comes with default styles that matches the other components' aesthetic."),
          AccordionItem("Is it animated?", "Yes. It's animated by default, but you can disable it if you prefer."),
          cls="w-96",
      )
    )
)
```

Try it out and let me know how it works!

In [ ]:
def AccordionItem(title, content, **kwargs):
    return Details(
        Summary(
            H2(
                title,
                ss("chevron-down", height=18, width=18, 
                       cls="text-muted-foreground pointer-events-none size-4 shrink-0 translate-y-0.5 transition-transform duration-200 group-open:rotate-180"),
                cls="flex flex-1 items-start justify-between gap-4 py-4 text-left text-sm font-medium hover:underline"
            ),
            cls="w-full focus-visible:border-ring focus-visible:ring-ring/50 focus-visible:ring-[3px] transition-all outline-none rounded-md"
        ),
        Section(
            P(content, cls="text-sm"),
            cls="pb-4"
        ),
        cls="group border-b last:border-b-0",
        **kwargs
    )

def Accordion(*items, **kwargs):
    return Div(
        Section(*items, cls="accordion"),
        **kwargs
    )

In [ ]:
p(
    Div(
        Accordion(
            AccordionItem("Is it accessible?", "Yes. It adheres to the WAI-ARIA design pattern."),
            AccordionItem("Is it styled?", "Yes. It comes with default styles that matches the other components' aesthetic."),
            AccordionItem("Is it animated?", "Yes. It's animated by default, but you can disable it if you prefer."),
            cls="w-96",
        ),
    cls="w-full h-96 flex items-center justify-center"
    )
)

### Alert

In [ ]:
AlertT = StrEnum("AlertT", {"default": "alert", "destructive": "alert-destructive"})

def Alert(title=None, description=None, icon=None, cls=AlertT.default, **kwargs):
    content = []
    if icon: content.append(icon)
    if title: content.append(H2(title))
    if description: content.append(Section(description))
    return Div(*content, cls=cls, **kwargs)

In [ ]:
p(
    Window(
        Alert(title="Warning!", icon=ss("circle-alert"), description="Task failed. Inspect element to retry?", cls=AlertT.destructive)
    )
)

### Button

In [ ]:
def kebab_to_snakecase(s, strip_s=""): return s.lstrip(strip_s).replace("-", "_")

In [ ]:
kebab_to_snakecase("btn-sm-icon-primary", "btn-")

'sm_icon_primary'

In [ ]:
def mk_cls_enum(enum_name, base, variants, modifiers, sizes):
    members = {}
    for sz, mod, var in product(sizes, modifiers, variants):
        val = f"{base}{sz}{mod}{var}"
        nm = kebab_to_snakecase(val, strip_s="-btn")
        members[nm] = val
    return StrEnum(enum_name, members)

In [ ]:
ButtonT = mk_cls_enum(
    "ButtonT", 
    "btn", 
    variants=["primary", "secondary", "destructive", "outline", "ghost", "link"], 
    modifiers=["", "icon-"], 
    sizes=["-", "-sm-", "-lg-"],
)

In [ ]:
ButtonT.sm_primary

<ButtonT.sm_primary: 'btn-sm-primary'>

In [ ]:
def Button(*args, cls=ButtonT.primary, **kwargs):
    return fh.Button(*args, cls=cls, **kwargs)

In [ ]:
p(
    Button("Testing", cls=ButtonT.lg_outline)
)

### Labels & Inputs

In [ ]:
def Input(*args, cls="", **kwargs):
    return fh.Input(*args, cls=f"input {cls}", **kwargs)

In [ ]:
p(
    Input(type="email", placeholder="Email")
)

In [ ]:
def Label(*args, cls="", **kwargs):
    return fh.Label(*args, cls=f"label {cls}", **kwargs)

In [ ]:
p(
    Label("Enter email")
)

In [ ]:
p(
    Div(
        Label("Email", _for="email"),
        Input(id="email", type="email", placeholder="Enter here...", disabled=True),
        cls="grid gap-3"
    )
)

### Avatar

In [ ]:
def Avatar(src:str, cls="", **kwargs):
    return Img(src=src, cls=f"size-8 shrink-0 object-cover rounded-full {cls}")

In [ ]:
p(
    Avatar(src="https://github.com/hunvreus.png")
)

### Breadcrumb

In [ ]:
def Breadcrumb(links:list, hrefs=None):
    if not hrefs: hrefs = [slugify(o) for o in links]
    contents = []
    for link, href in zip(links[:-1], hrefs):
        contents.append(Li(A(link, href=href, cls="hover:text-foreground transition-colors")))
        contents.append(Li(ss("chevron-right", cls="size-3.5")))
    contents.append(Li(Span(links[-1], cls="text-foreground font-normal")))

    return Ol(
        *contents,
        cls="text-muted-foreground flex flex-wrap items-center gap-1.5 text-sm break-words sm:gap-2.5",
    )

In [ ]:
p(
    Window(
        Breadcrumb(["Home", "Documents", "Components", "Breadcrumb"])
    )
)

### Badge

In [ ]:
BadgeT = StrEnum(
    "BadgeT",
    {
        'primary': 'badge-primary',
        'secondary': 'badge-secondary', 
        'destructive': 'badge-destructive',
        'outline': 'badge-outline'
    }
)

def Badge(*args, cls=BadgeT.primary, **kwargs):
    return Span(*args, cls=cls, **kwargs)

In [ ]:
p(
    Badge("primary"),
)

In [ ]:
p(
    A("Link", ss("arrow-right", height=12, width=12), cls=BadgeT.outline, href="/testing")
)

# Refs

In [ ]:
p(  
    Div(
        Div("TESTING", cls="card items-center w-lg mx-auto my-auto h-64"),
        cls="w-full h-96 flex flex-col my-auto"
    )
)

Okay that is working. I think let's try to make something and use that process to figure out the best approach.
I would like to make an example page that has a sidebar on the left with lots of buttons, an icon thing at the very bottom, and then in the main page some cards.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's start building this step by step. First, let me look at the Sidebar component to understand its structure:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01PWgb64LYwXZKD4spA7P4rA",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://basecoatui.com/components/sidebar"
    }
  },
  "result": "'[ Basecoat v0.3.6 ](/)\\n\\n### Getting started\\n\\n  * [ Introduction ](/introduction)\\n  * [ Install<TRUNCATED>"
}
```

</details>

Perfect! Now I understand the Sidebar structure. Based on what you want to build, here's what I see:

The Sidebar has:
- An `<aside class="sidebar">` wrapper
- A `<nav>` with sections containing navigation items
- Optional `<footer>` for the icon at the bottom
- A `<main>` for the content area

Would you like to start by creating the basic structure with just the outer containers first, or would you prefer to dive in and build a specific part (like the sidebar with buttons, or the main content with cards)?

Let's just try to build the sidebar. Please guide me through it, explaining the API I need to use.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Let's build the sidebar step by step.

From the docs, the basic structure is:

```html
<aside class="sidebar">
  <nav>
    <!-- navigation items go here -->
  </nav>
  <footer>
    <!-- optional footer content like your icon -->
  </footer>
</aside>
```

For the navigation items, Basecoat uses `<a class="item">` elements. You can add variants like `item-active` for the currently selected item.

Let's start with the outer structure. Try creating an `Aside` with class `"sidebar"` that contains a `Nav`. What do you get?

Can you give the examples to me in fasthtml style?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Of course! Here's the basic structure in FastHTML style:

```python
Aside(
    Nav(
        # navigation items go here
    ),
    Footer(
        # optional footer content like your icon
    ),
    cls="sidebar"
)
```

For the navigation items, you'd use:

```python
A("Dashboard", cls="item", href="#")
A("Settings", cls="item-active", href="#")  # for active item
```

Try creating a simple sidebar with a Nav containing a few A elements with the `"item"` class. What does it look like?

Sidebar docs:
Title: Sidebar | Basecoat

URL Source: https://basecoatui.com/components/sidebar/

Markdown Content:
[Usage](https://basecoatui.com/components/sidebar/#usage)
---------------------------------------------------------

### [HTML + JavaScript](https://basecoatui.com/components/sidebar/#usage-html-js)

#### [Step 1: Include the JavaScript files](https://basecoatui.com/components/sidebar/#usage-html-js-1)

You can either [include the JavaScript file for all the components](https://basecoatui.com/installation/#install-cdn-all), or just the one for this component by adding this to the `<head>` of your page:

```
<script src="https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/basecoat.min.js" defer></script>
<script src="https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/sidebar.min.js" defer></script>
```

[Components with JavaScript](https://basecoatui.com/installation/#install-js)[Use the CLI](https://basecoatui.com/installation/#install-cli)[sidebar.js](https://github.com/hunvreus/basecoat/blob/main/src/js/sidebar.js)

#### [Step 2: Add your sidebar HTML](https://basecoatui.com/components/sidebar/#usage-html-js-2)

```
<aside class="sidebar" data-side="left" aria-hidden="false">
  <nav aria-label="Sidebar navigation">
    <section class="scrollbar">
      <div role="group" aria-labelledby="group-label-content-1">
        <h3 id="group-label-content-1">Getting started</h3>

        <ul>
          <li>
            <a href="#">
              <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                <path d="m7 11 2-2-2-2" />
                <path d="M11 13h4" />
                <rect width="18" height="18" x="3" y="3" rx="2" ry="2" />
              </svg>
              <span>Playground</span>
            </a>
          </li>

          <li>
            <a href="#">
              <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                <path d="M12 8V4H8" />
                <rect width="16" height="12" x="4" y="8" rx="2" />
                <path d="M2 14h2" />
                <path d="M20 14h2" />
                <path d="M15 13v2" />
                <path d="M9 13v2" />
              </svg>
              <span>Models</span>
            </a>
          </li>

          <li>
            <details id="submenu-content-1-3">
              <summary aria-controls="submenu-content-1-3-content">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round">
                  <path d="M12.22 2h-.44a2 2 0 0 0-2 2v.18a2 2 0 0 1-1 1.73l-.43.25a2 2 0 0 1-2 0l-.15-.08a2 2 0 0 0-2.73.73l-.22.38a2 2 0 0 0 .73 2.73l.15.1a2 2 0 0 1 1 1.72v.51a2 2 0 0 1-1 1.74l-.15.09a2 2 0 0 0-.73 2.73l.22.38a2 2 0 0 0 2.73.73l.15-.08a2 2 0 0 1 2 0l.43.25a2 2 0 0 1 1 1.73V20a2 2 0 0 0 2 2h.44a2 2 0 0 0 2-2v-.18a2 2 0 0 1 1-1.73l.43-.25a2 2 0 0 1 2 0l.15.08a2 2 0 0 0 2.73-.73l.22-.39a2 2 0 0 0-.73-2.73l-.15-.08a2 2 0 0 1-1-1.74v-.5a2 2 0 0 1 1-1.74l.15-.09a2 2 0 0 0 .73-2.73l-.22-.38a2 2 0 0 0-2.73-.73l-.15.08a2 2 0 0 1-2 0l-.43-.25a2 2 0 0 1-1-1.73V4a2 2 0 0 0-2-2z" />
                  <circle cx="12" cy="12" r="3" />
                </svg>
                Settings
              </summary>
              <ul id="submenu-content-1-3-content">
                <li>
                  <a href="#">
                    <span>General</span>
                  </a>
                </li>

                <li>
                  <a href="#">
                    <span>Team</span>
                  </a>
                </li>

                <li>
                  <a href="#">
                    <span>Billing</span>
                  </a>
                </li>

                <li>
                  <a href="#">
                    <span>Limits</span>
                  </a>
                </li>
              </ul>
            </details>
          </li>
        </ul>
      </div>
    </section>
  </nav>
</aside>

<main>
  <button type="button" onclick="document.dispatchEvent(new CustomEvent('basecoat:sidebar'))">Toggle sidebar</button>
  <h1>Content</h1>
</main>
```

#### [HTML structure](https://basecoatui.com/components/sidebar/#usage-html-js-3)

`<aside class="sidebar" aria-hidden="false">`Wraps around the entire component. It can have the following attributes: 
*   `aria-hidden="true"`: controls the default state of the sidebar (hidden or visible).
*   `data-side="left"`: specifies the side of the sidebar (`left` or `right`, defaults to `left`).

`<nav>`
The navigation element that contains the sidebar's content. It can have the following attributes:

*   `id="{BUTTON_ID}"`: linked to by the `aria-labelledby` attribute of the listbox.
*   `aria-haspopup="menu"`: indicates that the button opens a menu.
*   `aria-controls="{ MENU_ID }"`: points to the menu's id.
*   `aria-expanded="false"`: tracks the popover's state.

`<header>`Optional The header of the sidebar.`<section>`The main navigation list. `<div role="group">`Group of options, can have a `aria-labelledby` attribute to link to a heading.`<span role="heading">`Group heading, must have an `id` attribute if you use the `aria-labelledby` attribute on the group.`<ul>`List of links or buttons. `<li>`Individual item. `<a>`A link. By default, clicking on a link will close the sidebar on mobile unless the `data-keep-mobile-sidebar-open` attribute is present.`<button>`A button. By default, clicking on a button will close the sidebar on mobile unless the `data-keep-mobile-sidebar-open` attribute is present.`<details>`Collapsible section. `<summary>`Summary of the collapsible section.`<ul>`List of links or buttons.`<footer>`Optional The footer of the sidebar.`<main>`A wrapper for the content of the page.`<button type="button" onclick="document.dispatchEvent(new CustomEvent('basecoat:sidebar'))">`A button to toggle the sidebar. If you want to use multiple sidebars you will need to add unique ids to the sidebars (i.e. `<aside class="sidebar" id="{SIDEBAR_ID}">`) and refer to them in the event `detail` (i.e. `document.dispatchEvent(new CustomEvent('basecoat:sidebar', { detail: { id: '{SIDEBAR_ID}' } }));`).
#### [JavaScript events](https://basecoatui.com/components/sidebar/#usage-html-js-4)

`basecoat:initialized`Once the component is fully initialized, it dispatches a custom (non-bubbling) `basecoat:initialized` event on itself.`basecoat:sidebar`
Sidebars listen for this event on `document` to open, close or toggle themselves. By default, the event will toggle the sidebar, but can be used to open or close if you add an `action` to the detail. Additionally, if you have multiple sidebars on the page, you can target a specific sidebar by adding its `id` to the detail:

```
<!-- Toggles the sidebar -->
<button type="button" onclick="document.dispatchEvent(new CustomEvent('basecoat:sidebar'));">Toggle sidebar</button>
<!-- Opens the `#main-navigation` sidebar -->
<button type="button" onclick="document.dispatchEvent(new CustomEvent('basecoat:sidebar', { detail: { id: 'main-navigation', action: 'open' } }));">Open sidebar</button>
<!-- Closes the sidebar -->
<button type="button" onclick="document.dispatchEvent(new CustomEvent('basecoat:sidebar', { detail: { action: 'close' } }));">Close sidebar</button>
```

### [Jinja and Nunjucks](https://basecoatui.com/components/sidebar/#usage-macro)

You can use the `sidebar()` Nunjucks or Jinja macro for this component.

[Use Nunjucks or Jinja macros](https://basecoatui.com/installation/#install-macros)[Jinja macro](https://github.com/hunvreus/basecoat/blob/main/src/jinja/sidebar.html.jinja)[Nunjucks macro](https://github.com/hunvreus/basecoat/blob/main/src/nunjucks/sidebar.njk)

```
{% set menu = [
  { type: "group", label: "Getting started", items: [
    { label: "Playground", url: "#" },
    { label: "Models", url: "#" },
    { label: "Settings", type: "submenu", items: [
      { label: "General", url: "#" },
      { label: "Team", url: "#" },
      { label: "Billing", url: "#" },
      { label: "Limits", url: "#" }
    ] }
  ]}
] %}

{{ sidebar(
  label="Sidebar navigation",
  menu=menu
) }}
<main>
  <h1>Content</h1>
</main>
```

In [ ]:
@rt
def index():
    return Div(
        Aside(
            Nav(
                "Nav items",
                Footer(
                    "footer content",
                ),
                aria_label="Sidebar navigation",
            ),
            cls="sidebar"
        ),
        cls="w-screen h-screen",
    )

In [ ]:
@rt
def index():
    return Div(
        Aside(
            Nav(
                Section(
                    Ul(
                        Li(A("Dashboard", href="#")),
                        Li(A("Settings", href="#")),
                        Li(A("Profile", href="#"))
                    )
                ),
                aria_label="Sidebar navigation"
            ),
            Footer(
                A("User Icon", href="#")
            ),
            cls="sidebar"
        ),
        cls="w-screen h-screen",
    )

Copying UkIcon from monsterui for now...

In [ ]:
def UkIcon(icon:str, # Icon name from [lucide icons](https://lucide.dev/icons/)
           height:int=None, 
           width:int=None, 
           stroke_width:int=None, # Thickness of lines
           cls=(), # Additional classes on the `Uk_icon` tag
           **kwargs # Additional args for `Uk_icon` tag
           )->FT: # a lucide icon of the specified size 
    "Creates an icon using lucide icons"
    return fh.Uk_icon(icon=icon, height=height, width=width, stroke_width=stroke_width, cls=cls, **kwargs)

In [ ]:
p(UkIcon("home"))

Okay, seems good. I'm going to try copy exactly an output from Shadcn. To begin with I'm going to extract some key components.
My current understanding is that there is a main Aside section with cls="sidebar", and then inside that we have a Nav section (always)?
The nav can then have things inside it like headers and the actual content. There seems to be different ways of doing collapsable groups, etc.

In [ ]:
def IconTitle(title:str, icon=None):
    return Div(UkIcon(icon), P(title), cls="flex items-center gap-2")

In [ ]:
p(
    IconTitle("Playground", icon="SquareTerminal")
)

In [ ]:
p(
    IconTitle("Playground")
)

In [ ]:
def Sidebar(*args, header=None, footer=None, **kwargs):
    return Aside(
        Nav(
            header,
            Section(*args, cls="scrollbar", **kwargs),
            footer,
        ),
        cls="sidebar",
    )

In [ ]:
def Group(title: str, *args, **kwargs):
    return Div(H3(title), *args, role="group", **kwargs)

In [ ]:
def SidebarGroup(title:str, name_list:list, icon_list:list, href_list=None):
    if not href_list: href_list = [f"/{o.lower().replace(" ", "-")}" for o in name_list]
    links = [Li(A(IconTitle(name, icon=ico), href=href)) for name, href, ico in zip(name_list, href_list, icon_list)]
    return Group(title, Ul(*links))


In [ ]:
def SidebarCollapsable(title:str, name_list:list, href_list=None):
    if not href_list: href_list = [f"/{o.lower().replace(" ", "-")}" for o in name_list]
    links = [Li(A(name, href=href)) for name, href in zip(name_list, href_list)]
    return Ul(Li(Details(Summary(title), Ul(*links))))

In [ ]:
def ToggleButton():
    return Button(
        UkIcon("panel-left"), 
        type="button",
        onclick="document.dispatchEvent(new CustomEvent('basecoat:sidebar'))",
        cls="btn-ghost p-2"
    )
p(ToggleButton())

In [ ]:
@rt
def index():
    return Div(
        Sidebar(
            Group(
                "Platform",
                SidebarCollapsable(IconTitle("Playground", icon="SquareTerminal"), ["History", "Starred", "Settings"]),
                SidebarCollapsable(IconTitle("Models", icon="Bot"), ["Genesis", "Explorer", "Quantum"]),
                SidebarCollapsable(IconTitle("Documentation", icon="BookOpen"), ["Introduction", "Get Started", "Tutorials", "Changelog"]),
                SidebarCollapsable(IconTitle("Settings", icon="Settings2"), ["General", "Team", "Billing", "Limits"]),
            ),
            SidebarGroup("Projects", ["Design Engineering", "Sales & Marketing", "Travel"], icon_list=["Frame", "PieChart", "Map"]),

        ),
        Main(
            ToggleButton(),
            H1("Main Content Area"),
            H3("Click Button above to open/close sidebar"),
            cls="p-8"
        ),
        cls="w-screen h-screen"
    )

### Dropdowns

### Context

```
Title: Dropdown Menu | Basecoat

URL Source: https://basecoatui.com/components/dropdown-menu/

Markdown Content:
Usage
-----

### [HTML + JavaScript](https://basecoatui.com/components/dropdown-menu/#usage-html-js)

#### [Step 1: Include the JavaScript files](https://basecoatui.com/components/dropdown-menu/#usage-html-js-1)

You can either [include the JavaScript file for all the components](https://basecoatui.com/installation/#install-cdn-all), or just the one for this component by adding this to the `<head>` of your page:

```
<script src="https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/basecoat.min.js" defer></script>
<script src="https://cdn.jsdelivr.net/npm/basecoat-css@0.3.6/dist/js/dropdown-menu.min.js" defer></script>
```

[Components with JavaScript](https://basecoatui.com/installation/#install-js)[Use the CLI](https://basecoatui.com/installation/#install-cli)[dropdown-menu.js](https://github.com/hunvreus/basecoat/blob/main/src/js/dropdown-menu.js)

#### [Step 2: Add your dropdown menu HTML](https://basecoatui.com/components/dropdown-menu/#usage-html-js-2)

```
<div id="demo-dropdown-menu" class="dropdown-menu">
  <button type="button" id="demo-dropdown-menu-trigger" aria-haspopup="menu" aria-controls="demo-dropdown-menu-menu" aria-expanded="false" class="btn-outline">Open</button>
  <div id="demo-dropdown-menu-popover" data-popover aria-hidden="true" class="min-w-56">
    <div role="menu" id="demo-dropdown-menu-menu" aria-labelledby="demo-dropdown-menu-trigger">
      <div role="group" aria-labelledby="account-options">
        <div role="heading" id="account-options">My Account</div>
        <div role="menuitem">
          Profile
          <span class="text-muted-foreground ml-auto text-xs tracking-widest">⇧⌘P</span>
        </div>
        <div role="menuitem">
          Billing
          <span class="text-muted-foreground ml-auto text-xs tracking-widest">⌘B</span>
        </div>
        <div role="menuitem">
          Settings
          <span class="text-muted-foreground ml-auto text-xs tracking-widest">⌘S</span>
        </div>
        <div role="menuitem">
          Keyboard shortcuts
          <span class="text-muted-foreground ml-auto text-xs tracking-widest">⌘K</span>
        </div>
      </div>
      <hr role="separator" />
      <div role="menuitem">GitHub</div>
      <div role="menuitem">Support</div>
      <div role="menuitem" aria-disabled="true">API</div>
      <hr role="separator" />
      <div role="menuitem">
        Logout
        <span class="text-muted-foreground ml-auto text-xs tracking-widest">⇧⌘P</span>
      </div>
    </div>
  </div>
</div>
```

#### [HTML structure](https://basecoatui.com/components/dropdown-menu/#usage-html-js-3)

`<div class="dropdown-menu">`Wraps around the entire component. `<button type="button" popovertarget="{ POPOVER_ID }">`
The trigger to open the popover, can also have the following attributes:

*   `id="{BUTTON_ID}"`: linked to by the `aria-labelledby` attribute of the listbox.
*   `aria-haspopup="menu"`: indicates that the button opens a menu.
*   `aria-controls="{ MENU_ID }"`: points to the menu's id.
*   `aria-expanded="false"`: tracks the popover's state.

`<div popover class="popover" id="{ POPOVER_ID }">`As with the [Popover](https://basecoatui.com/components/popover) component, you can set up the side and alignment of the popover using the `data-side` and `data-align` attributes. `<div role="menu">`The menu containing the options. Should have the following attributes: 
*   `id="{ MENU_ID }"`: refered to by the `aria-controls` attribute of the trigger.
*   `aria-labelledby="{ BUTTON_ID }"`: linked to by the button's `id` attribute.

`<button role="menuitem">`Menu item.`<button role="menuitemcheckbox">`Menu item with a checkbox.`<button role="menuitemradio">`Menu item with a radio button.`<hr role="separator">`Optional Separator between groups/options.`<div role="group">`Optional Group of options, can have a `aria-labelledby` attribute to link to a heading.`<span role="heading">`Group heading, must have an `id` attribute if you use the `aria-labelledby` attribute on the group.
#### [JavaScript events](https://basecoatui.com/components/dropdown-menu/#usage-html-js-4)

`basecoat:initialized`Once the component is fully initialized, it dispatches a custom (non-bubbling) `basecoat:initialized` event on itself.`basecoat:popover`When the popover opens, the component dispatches a custom (non-bubbling) `basecoat:popover` event on `document`. Other popover components (Combobox, Dropdown Menu, Popover and Select) listen for this to close any open popovers.
### [Jinja and Nunjucks](https://basecoatui.com/components/dropdown-menu/#usage-macro)

You can use the `dropdown_menu()` Nunjucks or Jinja macro for this component.

[Use Nunjucks or Jinja macros](https://basecoatui.com/installation/#install-macros)[Jinja macro](https://github.com/hunvreus/basecoat/blob/main/src/jinja/dropdown-menu.html.jinja)[Nunjucks macro](https://github.com/hunvreus/basecoat/blob/main/src/nunjucks/dropdown-menu.njk)

```
{% call dropdown_menu(
  id="dropdown-menu",
  trigger="Open",
  trigger_attrs={"class": "btn-outline"},
  popover_attrs={"class": "min-w-56"}
) %}
<div role="group" aria-labelledby="account-options">
  <div role="heading" id="account-options">My Account</div>
  <div role="menuitem">
    Profile
    <span class="text-muted-foreground ml-auto text-xs tracking-widest">⇧⌘P</span>
  </div>
  <div role="menuitem">
    Billing
    <span class="text-muted-foreground ml-auto text-xs tracking-widest">⌘B</span>
  </div>
  <div role="menuitem">
    Settings
    <span class="text-muted-foreground ml-auto text-xs tracking-widest">⌘S</span>
  </div>
  <div role="menuitem">
    Keyboard shortcuts
    <span class="text-muted-foreground ml-auto text-xs tracking-widest">⌘K</span>
  </div>
</div>
<hr role="separator">
<div role="menuitem">
  GitHub
</div>
<div role="menuitem">
  Support
</div>
<div role="menuitem" disabled>
  API
</div>
<hr role="separator">
<div role="menuitem">
  Logout
  <span class="text-muted-foreground ml-auto text-xs tracking-widest">⇧⌘P</span>
</div>
{% endcall %}
```

[Examples](https://basecoatui.com/components/dropdown-menu/#examples)
---------------------------------------------------------------------

### [Checkboxes](https://basecoatui.com/components/dropdown-menu/#example-checkboxes)

### [Radio Group](https://basecoatui.com/components/dropdown-menu/#example-radio-group)
```

### Code:

Basic sidepanel is complete.

Next steps:
- Logo button with dropdown popups for header + footer
- Sample breadcrumbing + default panels

In [ ]:
def DropdownMenu():
    return Div(
        Button("Open", id="demo-dropdown-trigger", cls="btn-outline"),
        Div(
            Div(
                Div(
                    Div("My Account", role="heading", id="account-options"),
                    Div("Profile", role="menuitem"),
                    role="group", aria_labelledby="account-options",
                ),
                role="menu", aria_labelledby="demo-dropdown-trigger",
            ),
            cls="min-w-65",
        ),
        cls="dropdown-menu",
    )

p(DropdownMenu())

Could you help give a minimal version of the dropdown menu example to help get it working?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code, you're very close! The issue is that the popover `Div` needs some specific attributes. Here's the minimal structure:

```python
def DropdownMenu():
    menu_id = "demo-dropdown-menu"
    return Div(
        Button("Open", 
               id=f"{menu_id}-trigger",
               aria_haspopup="menu",
               aria_controls=f"{menu_id}-content",
               aria_expanded="false",
               cls="btn-outline"),
        Div(
            Div(
                Div("Profile", role="menuitem"),
                Div("Settings", role="menuitem"),
                role="menu",
                id=f"{menu_id}-content",
                aria_labelledby=f"{menu_id}-trigger"
            ),
            data_popover=True,
            aria_hidden="true",
            cls="min-w-56"
        ),
        cls="dropdown-menu",
        id=menu_id
    )
```

The key missing pieces were `data_popover=True` and `aria_hidden="true"` on the popover div. Try this and see if it works!

In [ ]:
def DropdownMenu(name:str, menu_id="demo-dropdown-menu"):

    return Div(
        Button(name, 
               id=f"{menu_id}-trigger",
               aria_haspopup="menu",
               aria_controls=f"{menu_id}-content",
               aria_expanded="false",
               cls="btn-outline"),
        Div(
            Div(
                Div("Profile", role="menuitem"),
                Div("Settings", role="menuitem"),
                Hr(role="separator"),
                Div("Next thing", role="menuitem"),
                role="menu",
                id=f"{menu_id}-content",
                aria_labelledby=f"{menu_id}-trigger"
            ),
            data_popover=True,
            aria_hidden="true",
            cls="min-w-56"
        ),
        cls="dropdown-menu",
        id=menu_id
    )

In [ ]:
p(Div(DropdownMenu("Open"), cls="w-full h-64 p-8"))

Okay that looks good. What is the structure of groups and for sections (e.g., sectioned off by a line)?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at the docs, there are two ways to organize dropdown menu items:

**Groups** - Use `role="group"` with an optional heading:
```python
Div(
    Div("My Account", role="heading", id="account-options"),
    Div("Profile", role="menuitem"),
    Div("Billing", role="menuitem"),
    role="group",
    aria_labelledby="account-options"
)
```

**Separators** - Use `Hr` with `role="separator"`:
```python
Hr(role="separator")
```

So to create sections separated by lines, you'd do something like:
```python
Div("Profile", role="menuitem"),
Div("Settings", role="menuitem"),
Hr(role="separator"),
Div("Logout", role="menuitem")
```

Would you like to try adding some groups or separators to your dropdown menu?

In [ ]:
@rt
def index():
    return Div(
        Sidebar(
            Group(
                "Platform",
                SidebarCollapsable(IconTitle("Playground", icon="SquareTerminal"), ["History", "Starred", "Settings"]),
                SidebarCollapsable(IconTitle("Models", icon="Bot"), ["Genesis", "Explorer", "Quantum"]),
                SidebarCollapsable(IconTitle("Documentation", icon="BookOpen"), ["Introduction", "Get Started", "Tutorials", "Changelog"]),
                SidebarCollapsable(IconTitle("Settings", icon="Settings2"), ["General", "Team", "Billing", "Limits"]),
            ),
            SidebarGroup("Projects", ["Design Engineering", "Sales & Marketing", "Travel"], icon_list=["Frame", "PieChart", "Map"]),

        ),
        Main(
            ToggleButton(),
            H1("Main Content Area"),
            H3("Click Button above to open/close sidebar"),
            P("Testing dropdowns:"),
            DropdownMenu("Open"),
            cls="p-8"
        ),
        cls="w-screen h-screen"
    )